In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import os
import torchvision.models as models
from torchvision import transforms
import pandas as pd
from torch.utils.data.sampler import WeightedRandomSampler
import random
import math
from models.resnet_cbam import resnet50_cbam

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16
writer = SummaryWriter('runs/mobileNetV2')

In [3]:
csv = pd.read_csv('./images/dataset/0_index.csv',usecols=[0,2,4])
csv_numpy = csv.to_numpy()
#csv2 = pd.read_csv('./images/processed/0_index.csv',usecols=[0,2,4])
#csv_numpy2 = csv2.to_numpy()
#csv3 = pd.read_csv('./images/gabor_processed/0_index.csv',usecols=[0,2,4])
#csv_numpy3 = csv3.to_numpy()
#这是同一个文件

In [4]:
ss1,DD1,sD1,Ds1 = [],[],[],[]
ss2,DD2,sD2,Ds2 = [],[],[],[]
ss3,DD3,sD3,Ds3 = [],[],[],[]
for i in range(len(csv_numpy)):
    if csv_numpy[i][1] == 's/s':
        ss1.append('./images/dataset/' + csv_numpy[i][0])
        ss2.append('./images/processed/' + csv_numpy[i][0])
        ss3.append('./images/gabor_processed/G_' + csv_numpy[i][0])
    elif csv_numpy[i][1] == 'D/D':
        DD1.append('./images/dataset/' + csv_numpy[i][0])
        DD2.append('./images/processed/' + csv_numpy[i][0])
        DD3.append('./images/gabor_processed/G_' + csv_numpy[i][0])
    elif csv_numpy[i][1] == 'D/s':
        Ds1.append('./images/dataset/' + csv_numpy[i][0])
        Ds2.append('./images/processed/' + csv_numpy[i][0])
        Ds3.append('./images/gabor_processed/G_' + csv_numpy[i][0])
    elif csv_numpy[i][1] == 's/D':
        sD1.append('./images/dataset/' + csv_numpy[i][0])
        sD2.append('./images/processed/' + csv_numpy[i][0])
        sD3.append('./images/gabor_processed/G_' + csv_numpy[i][0])
random.seed(18252589459)
random.shuffle(ss2)
random.shuffle(DD2)
random.shuffle(Ds2)
random.shuffle(sD2)
print(len(ss2),len(DD2),len(Ds2),len(sD2))

225 188 41 31


In [5]:
train_ratio = 0.70
test_ratio = 0.30
train_set1, test_set1 = [],[]
train_set2, test_set2 = [],[]
train_set3, test_set3 = [],[]
#这四个for以后想想能不能优化一下
for i in range(len(ss1)):
    temp1 = [ss1[i],0]
    temp2 = [ss2[i],0]
    temp3 = [ss3[i],0]
    if i<int(train_ratio*len(ss1)):
        #train_set1.append(temp1)
        train_set2.append(temp2)
        #train_set3.append(temp3)
    else:
        #test_set1.append(temp1)
        test_set2.append(temp2)
        #test_set3.append(temp3)
for i in range(len(DD1)):
    temp1 = [DD1[i],1]
    temp2 = [DD2[i],1]
    temp3 = [DD3[i],1]
    if i<int(train_ratio*len(DD1)):
        #train_set1.append(temp1)
        train_set2.append(temp2)
        #train_set3.append(temp3)
    else:
        #test_set1.append(temp1)
        test_set2.append(temp2)
        #test_set3.append(temp3)
for i in range(len(Ds1)):
    temp1 = [Ds1[i],2]
    temp2 = [Ds2[i],2]
    temp3 = [Ds3[i],2]
    if i<int(train_ratio*len(Ds1)):
        #train_set1.append(temp1)
        train_set2.append(temp2)
        #train_set3.append(temp3)
    else:
        #test_set1.append(temp1)
        test_set2.append(temp2)
        #test_set3.append(temp3)
for i in range(len(sD1)):
    temp1 = [sD1[i],3]
    temp2 = [sD2[i],3]
    temp3 = [sD3[i],3]
    if i<int(train_ratio*len(sD1)):
        #train_set1.append(temp1)
        train_set2.append(temp2)
        #train_set3.append(temp3)
    else:
        #test_set1.append(temp1)
        test_set2.append(temp2)
        #test_set3.append(temp3)
        
train_set = train_set1+train_set2+train_set3
test_set = test_set1+test_set2+test_set3
print(len(train_set),len(test_set))

337 148


In [6]:
class MyDataset(Dataset):
    def __init__(self, dataset, train, transform=None, target_transform=None):
        imgs = []  # 内部为多个tuple，每个tuple为(图片路径，label)
        if train:
            for data in dataset:
                for i in range(3):#训练集数据复制三份
                    imgs.append((data[0],data[1]))
        else:
            for data in dataset:
                imgs.append((data[0],data[1]))            
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        fn, label = self.imgs[index]
        img = Image.open(fn).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.imgs)

In [20]:
img = Image.open(ss1[0]).convert('RGB')
tR = transforms.RandomCrop(896)
tF = transforms.FiveCrop(896)
tFlist = []
for i in tF(img):
    tFlist.append(i)
a = transforms.ToTensor()(tR(img))
a.shape

torch.Size([3, 896, 896])

In [11]:
train_transforms111 = transforms.Compose([
    transforms.RandomCrop(896),
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    #transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
img = Image.open(ss1[0]).convert('RGB')
img_trans = train_transforms111(img)
writer.add_image('test_image2',img_trans)
#结论：必须是经过ToTensor的图片才能写入writer

In [18]:
writer.add_graph(model_conv,torch.rand(32,3,3,3))#输入模型和输入尺寸（用于内部函数正确遍历网络），我目前获得尺寸的方法是看尺寸输入错误时的报错信息。。。

In [7]:
train_transforms = transforms.Compose([
    transforms.RandomCrop(896),
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    #transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    #transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
'''test_transforms = transforms.Compose([
    transforms.Resize(896),
    transforms.Grayscale(num_output_channels=3),
    transforms.FiveCrop(224),#不用tencrop了，感觉再增加更多相似图片，没啥用
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
    transforms.Lambda(lambda crops: torch.stack([transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(crop) for crop in crops])),
])'''

'test_transforms = transforms.Compose([\n    transforms.Resize(896),\n    transforms.Grayscale(num_output_channels=3),\n    transforms.FiveCrop(224),#不用tencrop了，感觉再增加更多相似图片，没啥用\n    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),\n    transforms.Lambda(lambda crops: torch.stack([transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(crop) for crop in crops])),\n])'

In [8]:
train_data = MyDataset(train_set, train=True, transform=train_transforms)
test_data = MyDataset(test_set, train=False, transform=test_transforms)

weights = []
for data,target in train_data.imgs:
    if target == 2:
        weights.append(4)
    elif target == 3:
        weights.append(5)
    else:
        weights.append(1)
sampler = WeightedRandomSampler(weights, num_samples=1500, replacement=True)

# 然后就是调用DataLoader和刚刚创建的数据集，来创建dataloader，这里提一句，loader的长度是有多少个batch，所以和batch_size有关
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, sampler=sampler, num_workers=0)
#train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
print('num_of_trainData:', len(train_loader))
print('num_of_testData:', len(test_loader))

num_of_trainData: 94
num_of_testData: 10


In [9]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        x, y = data
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_hat = model(x)
        loss = criterion(y_hat, y)
        train_loss += criterion(y_hat, y).item()
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader.dataset)
    train_loss *= 100.0
    writer.add_scalar('training loss', train_loss,epoch)
    print('Train Epoch: {}\t Average Loss: {:.6f}'.format(epoch, train_loss))


def test(model, device, test_loader,epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            x, y = data
            #bs, nc, c, h, w = x.size()
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            y_hat = model(x)
            #y_hat_all = model(x.view(-1,c,h,w))
            #y_hat = y_hat_all.view(bs,nc,-1).mean(1)#求5个crop的平均预测结果
            test_loss += criterion(y_hat, y).item()  # sum up batch loss
            pred = y_hat.max(1, keepdim=True)[1]  # get the index of the max log-probability
            correct += pred.eq(y.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_loss *= 100.0
    writer.add_scalar('test accuary', 100. * correct / len(test_data) ,epoch)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.6f}%)\n'.format(
        test_loss, correct, len(test_data),
        100. * correct / len(test_data)))

In [13]:
model_resnest = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
'''for param in model_resnest.parameters():
    param.requires_grad = False
    
# Parameters of newly constructed modules have requires_grad=True by default    
for param in model_resnest.layer4[2].conv3.parameters():
    param.requires_grad = True'''
model_resnest.fc = nn.Linear(2048,4)

model_resnest = model_resnest.to(DEVICE)

# criterion = nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss()

# criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {'params': model_resnest.parameters()},
    #{'params': model_resnest.layer4[2].conv3.parameters()},
], lr=0.001)

Using cache found in /home/yyh/.cache/torch/hub/zhanghang1989_ResNeSt_master


RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 10.92 GiB total capacity; 1.59 GiB already allocated; 2.50 MiB free; 1.68 GiB reserved in total by PyTorch)

In [12]:
for epoch in range(1, 200):
    train(model=model_resnest, device=DEVICE, train_loader=train_loader, optimizer = optimizer, epoch=epoch)
    test(model=model_resnest, device=DEVICE, test_loader=test_loader, epoch=epoch)

RuntimeError: CUDA out of memory. Tried to allocate 14.00 MiB (GPU 0; 10.92 GiB total capacity; 1.59 GiB already allocated; 2.50 MiB free; 1.68 GiB reserved in total by PyTorch)

#----------------------------------------------------------

In [34]:
model_cbam = resnet50_cbam(pretrained=True)
#torch.hub.list('zhanghang1989/ResNeSt', force_reload=True)
model_resnest = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
model_resnext = torch.hub.load('pytorch/vision:v0.6.0', 'resnext50_32x4d', pretrained=True)

In [10]:
model_conv = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
for param in model_conv.features[17].conv[2].parameters():
    param.requires_grad = True
for param in model_conv.features[18][0].parameters():
    param.requires_grad = True
num_ftrs = model_conv.classifier[1].in_features
model_conv.classifier[1] = nn.Linear(num_ftrs, 4)
model_conv = model_conv.to(DEVICE)

criterion = nn.CrossEntropyLoss()

# criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {'params': model_conv.features[17].conv[2].parameters()},
    {'params': model_conv.features[18][0].parameters()},
    {'params': model_conv.classifier[1].parameters()},
    #{'params': model_conv.parameters()},
], lr=0.001)

Using cache found in /home/yyh/.cache/torch/hub/pytorch_vision_v0.6.0


In [16]:
model_conv = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2')


num_ftrs = model_conv.classifier[1].in_features
model_conv.classifier[1] = nn.Linear(num_ftrs, 4)
model_conv = model_conv.to(DEVICE)

# criterion = nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss()

# criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {'params': model_conv.parameters()},
], lr=0.001)

Using cache found in /home/yyh/.cache/torch/hub/pytorch_vision_v0.6.0


TypeError: 'NoneType' object is not iterable

In [11]:
for epoch in range(1, 100):
    train(model=model_conv, device=DEVICE, train_loader=train_loader, optimizer = optimizer, epoch=epoch)
    test(model=model_conv, device=DEVICE, test_loader=test_loader, epoch=epoch)

Train Epoch: 1	 Average Loss: 6.690472

Test set: Average loss: 4.7033, Accuracy: 113/148 (76.351351%)

Train Epoch: 2	 Average Loss: 5.292434

Test set: Average loss: 4.3483, Accuracy: 118/148 (79.729730%)

Train Epoch: 3	 Average Loss: 4.170250

Test set: Average loss: 4.5679, Accuracy: 111/148 (75.000000%)

Train Epoch: 4	 Average Loss: 3.965270

Test set: Average loss: 6.0847, Accuracy: 108/148 (72.972973%)

Train Epoch: 5	 Average Loss: 3.990326

Test set: Average loss: 3.8060, Accuracy: 118/148 (79.729730%)

Train Epoch: 6	 Average Loss: 3.542422

Test set: Average loss: 4.9587, Accuracy: 105/148 (70.945946%)

Train Epoch: 7	 Average Loss: 3.276531

Test set: Average loss: 3.7587, Accuracy: 122/148 (82.432432%)

Train Epoch: 8	 Average Loss: 3.433023

Test set: Average loss: 4.3743, Accuracy: 122/148 (82.432432%)

Train Epoch: 9	 Average Loss: 3.155132

Test set: Average loss: 4.3057, Accuracy: 121/148 (81.756757%)

Train Epoch: 10	 Average Loss: 3.172202

Test set: Average loss:

KeyboardInterrupt: 

In [13]:
img = Image.open(ss1[0]).convert('RGB')
img_trans = test_transforms(img)
out = model_conv(img_trans)
out

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [32, 3, 3, 3], but got 3-dimensional input of size [3, 224, 224] instead

In [ ]:
def images_to_probs(net, images):
    '''
    Generates predictions and corresponding probabilities from a trained
    network and a list of images
    '''
    output = net(images)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.numpy())
    return preds, [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]


def plot_classes_preds(net, images, labels):
    '''
    Generates matplotlib Figure using a trained network, along with images
    and labels from a batch, that shows the network's top prediction along
    with its probability, alongside the actual label, coloring this
    information based on whether the prediction was correct or not.
    Uses the "images_to_probs" function.
    '''
    preds, probs = images_to_probs(net, images)
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(12, 48))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        matplotlib_imshow(images[idx], one_channel=True)
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            classes[preds[idx]],
            probs[idx] * 100.0,
            classes[labels[idx]]),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig